# Zmiana formatu na użytkownik/produkt/ocena

In [1]:
import pandas as pd

In [2]:
file_path = 'preferencje.xlsx'
excel_data = pd.ExcelFile(file_path)

# Check sheet names to understand the structure
excel_data.sheet_names

['Arkusz1']

In [3]:
sheet_data = pd.read_excel(file_path, sheet_name='Arkusz1')
sheet_data.head()

,Unnamed: 0,https://hiszpanskiesmaki.pl/przepis-na-paella-mixta-z-owocami-morza-i-miesem/,https://www.kwestiasmaku.com/przepis/pascha-wielkanocna,https://aniagotuje.pl/przepis/rosol-z-kury,https://www.kwestiasmaku.com/przepis/zupa-z-dyni,https://www.kwestiasmaku.com/przepis/salatka-gyros,https://aniagotuje.pl/przepis/poledwiczki-w-sosie-musztardowym,Kaszanka z grilla | AniaGotuje.pl,Sałatka z ananasem | AniaGotuje.pl,https://www.mniammniam.com/przepis-na-miesny-jez,...,https://aniagotuje.pl/przepis/zurek-wielkanocny,https://aniagotuje.pl/przepis/kebab,https://aniagotuje.pl/przepis/tatar,https://aniagotuje.pl/przepis/falafel,https://aniagotuje.pl/przepis/tradycyjny-barszcz-wigilijny,https://aniagotuje.pl/przepis/pierogi-ruskie,https://www.doradcasmaku.pl/przepis-kiszka-ziemniaczana-258674,https://aniagotuje.pl/przepis/strogonow,https://aniagotuje.pl/przepis/nalesniki-z-serem-i-owocami,https://www.kwestiasmaku.com/kuchnia_polska/pierogi/pierogi_z_miesem/przepis.html
0,NaN,paella,pascha wielkanocna,rosół z kury,zupa dyniowa,Gyros,Polędwiczki w sosie musztardowym,Kaszanka,Sałatka z ananasem,Mięsny jeż,...,żurek,kebab,tatar,falafel,barszcz czerwony,pierogi ruskie,kiszka ziemniaczana,Boeuf strogonow,Naleśniki z serem,Pierogi z mięsem
1,Users/Items,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ula,4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,109815,3,3,5,5,4,3,2,5,4,...,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,109521,4,5,4,3,5,5,4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data = pd.read_excel(file_path, sheet_name='Arkusz1', skiprows=1) 

# Drop empty columns and rows to clean the data
data_cleaned = data.dropna(how="all", axis=1).dropna(how="all", axis=0)

# Rename columns for clarity
data_cleaned.columns = ["user"] + list(data_cleaned.columns[1:])

# Melt the dataframe to transform it into user/product/score format
data_melted = pd.melt(data_cleaned, id_vars=["user"], var_name="product", value_name="rating")

# Drop rows where Score is NaN (unrated products)
data_final = data_melted.dropna(subset=["rating"])

data_final

,user,product,rating
1,Ula,paella,4.0
2,109815,paella,3.0
3,109521,paella,4.0
4,107792,paella,3.0
6,109812,paella,4.0
...,...,...,...
395,109852,Boeuf strogonow,5.0
396,107 885,Boeuf strogonow,3.0
417,107 885,Naleśniki z serem,5.0
438,107 885,Pierogi z mięsem,4.0


In [5]:
data_final.to_csv("data.csv",index=False)

# Ocena algorytmów

In [20]:
import pandas as pd
from surprise import Dataset, Reader
from surprise import SVD, KNNBasic, BaselineOnly, NormalPredictor,Random
from surprise.model_selection import cross_validate


data = pd.read_csv("data.csv")  # Upewnij się, że plik zawiera odpowiednie kolumny
reader = Reader(rating_scale=(1.0, 5.0))  # Ustaw zakres ocen, np. od 1 do 5
dataset = Dataset.load_from_df(data[["user", "product", "rating"]], reader)

algo = SVD()
cross_validate(algo, dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3558  1.1494  1.3608  1.2083  1.2199  1.2588  0.0847  
MAE (testset)     1.0369  0.9588  1.1318  0.9825  0.9719  1.0164  0.0635  
Fit time          0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([1.35583141, 1.14941491, 1.36079631, 1.20825156, 1.21988753]),
 'test_mae': array([1.03690485, 0.95878265, 1.13175966, 0.98249825, 0.97193879]),
 'fit_time': (0.0, 0.0, 0.0010004043579101562, 0.0, 0.0),
 'test_time': (0.001008749008178711,
  0.00099945068359375,
  0.0,
  0.0,
  0.0010018348693847656)}

In [24]:
algo = NormalPredictor()
cross_validate(algo, dataset, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5124  1.8194  1.8617  1.8058  1.7569  1.7512  0.1240  
Fit time          0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([1.51240825, 1.81939707, 1.86169091, 1.80583615, 1.75688096]),
 'fit_time': (0.0,
  0.001013040542602539,
  0.0004980564117431641,
  0.0003859996795654297,
  0.0),
 'test_time': (0.0, 0.0002040863037109375, 0.0, 0.0, 0.0)}

In [23]:
algo = KNNBasic()
cross_validate(algo, dataset, measures=['RMSE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2864  1.2670  1.0459  1.2316  1.1951  1.2052  0.0856  
Fit time          0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([1.28635829, 1.26703903, 1.04585144, 1.23164524, 1.1951414 ]),
 'fit_time': (0.0, 0.000499725341796875, 0.0, 0.0, 0.0),
 'test_time': (0.0010688304901123047,
  0.0,
  0.0011818408966064453,
  0.0,
  0.00101470947265625)}

In [25]:
algo = BaselineOnly()
cross_validate(algo, dataset, measures=['RMSE'], cv=5, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3129  1.2531  1.1897  1.2171  1.1534  1.2253  0.0547  
Fit time          0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([1.31290542, 1.25314944, 1.18972269, 1.21707945, 1.15340215]),
 'fit_time': (0.0010039806365966797,
  0.0005555152893066406,
  0.0005116462707519531,
  0.0,
  0.00016498565673828125),
 'test_time': (0.0, 0.0, 0.0, 0.0, 0.0)}

In [37]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

# Przygotowanie danych
reader = Reader(rating_scale=(1, 5))  # Określenie zakresu ocen
data123 = Dataset.load_from_df(data[["user", "product", "rating"]], reader)

# Podział danych na zbiór treningowy i testowy
trainset, testset = train_test_split(data123, test_size=0.2)

# Algorytm rekomendacji
algo = SVD()  # Można użyć innego algorytmu np. KNNBasic, RandomPredictor, itp.
algo.fit(trainset)

# Wybrany użytkownik
selected_user = '107885'  # Zamień na ID użytkownika, którego rekomendacje chcesz wygenerować

# Lista wszystkich produktów
all_products = data['product'].unique()

# Produkty, które użytkownik już ocenił
rated_products = data[data['user'] == selected_user]['product'].values

# Produkty, których użytkownik jeszcze nie ocenił
unrated_products = [p for p in all_products if p not in rated_products]

# Przewidywanie ocen dla nieocenionych produktów
recommendations = []
for product in unrated_products:
    prediction = algo.predict(selected_user, product)
    recommendations.append((product, prediction.est))

# Sortowanie rekomendacji według przewidywanej oceny (malejąco)
recommendations.sort(key=lambda x: x[1], reverse=True)

print(f"Trzy najbardziej rekomendowane potrawy dla {selected_user}:")
for i in range(min(3, len(recommendations))):
    product, rating = recommendations[i]
    print(f"{i + 1}. {product}, ocena {rating:.2f}")


Trzy najbardziej rekomendowane potrawy dla 107885:
1. kebab, ocena 4.08
2. ciasto murzynek, ocena 4.04
3. Gyros, ocena 3.99


Trzy najbardziej rekomendowane potrawy dla 107885:
1. kebab z przewidywaną oceną 4.17
2. ciasto murzynek z przewidywaną oceną 4.11
3. żurek z przewidywaną oceną 4.00
